!pip install beautifulsoup4 #install

!pip install nltk #install

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

In [ ]:
import matplotlib
matplotlib.use('Agg')
#matplotlib.rcParams['backend'] = "GtkAgg"

import matplotlib.pyplot as plt

In [ ]:
import pyspark
sc = pyspark.SparkContext('local[*]')


In [ ]:
from pyspark import SparkContext
from pyspark.mllib.feature import HashingTF
import csv

Taking only Sentiments as Integers

In [ ]:
sentiments = sc.textFile("./data/train.csv").map(lambda line: line.split(",",2)[0].strip("\"")).map(int)

In [ ]:
sentiments_test = sc.textFile("./data/test.csv").map(lambda line: line.split(",",2)[0].strip("\"")).map(int)

Below is the Text Processing Function taken from our Data Mining Research Project

In [ ]:
import re


from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup

def ProcessTweet(Tweet):
    try:
        #Tweet = Tweet.encode('utf-8')
        stemmer = PorterStemmer()
        print (Tweet)
        Tweet = Tweet.lower()
        soup = BeautifulSoup(Tweet, "html.parser")
        eTags= soup.find_all('e')
        for tag in eTags:
            tag.extract()
        #Tweet = soup.encode('ascii')
        Tweet = re.sub(r'pic.twitter.*?$', '', Tweet)
        Tweet = re.sub(r'pic.twitter.*? ', '', Tweet)
        Tweet = re.sub(r'(.)\1{2,}', r'\1\1', Tweet)
        Tweet = re.sub('((www\.[^\s]+)|(http://[^\s]+))','',Tweet)  #remove url
        Tweet = re.sub(r'#([^\s])*$', '', Tweet)
        Tweet = re.sub(r'#([^\s])* ', '', Tweet)
        Tweet = re.sub(r'\@([^\s])*$', '', Tweet)
        Tweet = re.sub(r'\@([^\s])* ', '', Tweet)
        Tweet = re.sub("\d",'',Tweet)                               #remove digits
        Tweet = re.sub("[!\'.\"%/*$;:\(\):,?]",'',Tweet)                        #remove special characters
        Tweet = re.sub(r'\-',' ',Tweet)                             #replace - with white space
        Tweet = re.sub(r'\'m', ' am', Tweet)
        Tweet = re.sub(r'\'d', ' would', Tweet)
        Tweet = re.sub(r'\'ll', ' will', Tweet)
        Tweet = re.sub(r'\&', 'and', Tweet)
        Tweet = re.sub(r'\b\w{1,2}\b','', Tweet).strip()            #remove 2 or lesser length words
        Tweet = re.sub('[\s]+', ' ', Tweet)
        Tweet = re.sub('<>', '', Tweet)
        print (Tweet)
        #remove trailing duplicate characters
        words=[]
        for word in Tweet.split():
            l=word[-1]
            word = word.rstrip(word[-1])
            word += l
            words.append(word)
        Tweet = ' '.join(words)
        print (Tweet)
        #stemming
        words=[]
        for word in Tweet.split():
            data = stemmer.stem(word)
            words.append(str(data))
        Tweet = ' '.join(words)
        print (Tweet)
        #stop word removal
        stop = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 
                'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs',
                'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were',
                'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 
                'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 
                'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 
                'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 
                'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 
                'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now']
        final = [i for i in Tweet.split() if i not in stop]
        tweet = ' '.join(final)
        print (tweet)
        return tweet
    except Exception as e:
        print(e)

Sending only the Tweets to the Processing Function

In [ ]:
tweets = sc.textFile("./data/train.csv",).map(lambda line: line.split(",",5)[-1]).map(ProcessTweet)

In [ ]:
tweets_test = sc.textFile("./data/test.csv",).map(lambda line: line.split(",",5)[-1]).map(ProcessTweet)

Sample Tweets after Processing

In [ ]:
tweets = tweets.map(lambda line: line.split())
tweets.take(5)

In [ ]:
tweets_test = tweets_test.map(lambda line: line.split())

Extracting Features using the mllib package

In [ ]:
hashingTF = HashingTF()
tf = hashingTF.transform(tweets)

In [ ]:
hashingTF1 = HashingTF()
tf_test = hashingTF1.transform(tweets_test)

In [ ]:
tf.cache()

In [ ]:
tf_test.cache()

In [ ]:
from pyspark.mllib.feature import IDF
idf = IDF(minDocFreq=0).fit(tf)

Transforming Training and Test data features to tf-idf. For test data, IDF of train data is used.

In [ ]:
tfidf = idf.transform(tf)

In [ ]:
tfidf_test = idf.transform(tf_test)

In [ ]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import NaiveBayes

Concatenating Sentiments and Features as labeled point for modeling

In [ ]:
training = sentiments.zip(tfidf).map(lambda x: LabeledPoint(x[0],x[1]))

In [ ]:
test = sentiments_test.zip(tfidf_test).map(lambda x: LabeledPoint(x[0],x[1]))

In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics

Run All Cells above this one. From here on, use the cell you want to exectue

#Naive bayes Model. 

In [ ]:

model = NaiveBayes.train(training)

#Can change the 'test' to 'train' to predict accordingly
labelsAndPred = test.map(lambda p: (float(model.predict(p.features)),p.label))
from pyspark.mllib.evaluation import BinaryClassificationMetrics
#metrics object instantiation
metrics = MulticlassMetrics(labelsAndPred)
metrics2  = BinaryClassificationMetrics(labelsAndPred)


In [ ]:
# Calculating metrics from the confusion matrix
x = metrics.confusionMatrix().toArray()
trueP=x[1][1] 
trueN=x[0][0]
falseP=x[0][1]
falseN=x[1][0]
NBprecision= trueP/(trueP+falseP)
NBrecall= trueP/(trueP+falseN)
NBf1score= (2*NBprecision*NBrecall)/(NBprecision+NBrecall)
NBaccuracy = (trueP+trueN)/(trueP+trueN+falseP+falseN)
print (NBprecision,NBrecall,NBf1score,NBaccuracy)

10-fold Cross Validation for Naive Bayes

In [ ]:
#tprecision= 0
#trecall= 0
#tf1score1= 0
taccuracy = 0
for i in range(10):
    
    trainrdd, testrdd = training.randomSplit([0.9,0.1], 2*i*i)
    model = NaiveBayes.train(trainrdd)
    labelsAndPred = testrdd.map(lambda p: (float(model.predict(p.features)),p.label))
    metrics = MulticlassMetrics(labelsAndPred)
    x = metrics.confusionMatrix().toArray()
    trueP=x[1][1]
    trueN=x[0][0]
    falseP=x[0][1]
    falseN=x[1][0]
    #tprecision+= trueP/(trueP+falseP)
    #trecall+= trueP/(trueP+falseN)
    #tf1score1+= (2*tprecision*trecall)/(tprecision+trecall)
    taccuracy+= (trueP+trueN)/(trueP+trueN+falseP+falseN)
    print(taccuracy)
accuracy = taccuracy/10
print("Final Accuracy for K-Cross Naive bayes is: ",accuracy)

Attempt to computer ROC for NB

In [ ]:
tpr = []
fpr = []
for lm in range(11):
    model = NaiveBayes.train(training, lambda_=lm/10.0)
    labelsAndPred = test.map(lambda p: (float(model.predict(p.features)),p.label))
    from pyspark.mllib.evaluation import BinaryClassificationMetrics
    metrics = MulticlassMetrics(labelsAndPred)
    metrics2  = BinaryClassificationMetrics(labelsAndPred)
    x = metrics.confusionMatrix().toArray()
    trueP=x[1][1] 
    trueN=x[0][0]
    falseP=x[0][1]
    falseN=x[1][0]
    NBrecall= trueP/(trueP+falseN)
    tpr.append(NBrecall)
    NBfpr= falseP/(falseP+trueN)
    fpr.append(NBfpr)
    print(tpr,fpr)

In [ ]:
print (metrics2.areaUnderROC)
print()

Logistic Regression

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
model = LogisticRegressionWithLBFGS.train(training,numClasses=2)
#Can change the 'test' to 'train' to predict accordingly
labelsAndPred = test.map(lambda p: (float(model.predict(p.features)),p.label))


In [ ]:
# Calculating metrics from the confusion matrix
metrics = MulticlassMetrics(labelsAndPred)
x = metrics.confusionMatrix().toArray()
trueP=x[1][1]
trueN=x[0][0]
falseP=x[0][1]
falseN=x[1][0]
logprecision= trueP/(trueP+falseP)
logrecall= trueP/(trueP+falseN)
logfpr=
logf1score= (2*logprecision*logrecall)/(logprecision+logrecall)
logaccuracy = (trueP+trueN)/(trueP+trueN+falseP+falseN)
print (logprecision,logrecall,logf1score,logaccuracy)

Calculating the ROC by changing the threshold values for LOG from 0.0-1.0

In [ ]:
model = LogisticRegressionWithLBFGS.train(training,numClasses=2)
tpr = []
fpr = []
for j in range(11):    
    model.setThreshold(j/10.0)
    labelsAndPred = test.map(lambda p: (float(model.predict(p.features)),p.label))
    metrics = MulticlassMetrics(labelsAndPred)
    x = metrics.confusionMatrix().toArray()
    trueP=x[1][1]
    trueN=x[0][0]
    falseP=x[0][1]
    falseN=x[1][0]
    logrecall= trueP/(trueP+falseN)
    tpr.append(logrecall)
    logfpr= falseP/(falseP+trueN)
    fpr.append(logfpr)
roc_auc = auc(fpr, tpr)

Plotting The ROC for LOG

In [ ]:
import matplotlib.pyplot as plt2
plt2.clf()
plt2.plot(fpr, tpr, lw=1, label='ROC (area = %0.2f)' % (roc_auc))
plt2.xlim([-0.05, 1.05])
plt2.ylim([-0.05, 1.05])
plt2.xlabel('False Positive Rate')
plt2.ylabel('True Positive Rate')
plt2.title('Receiver operating characteristic')
plt2.legend(loc="lower right")
plt2.show()
plt2.savefig('ROC-LOG.png') #savefig in home directory if .show() does not work

10-fold Cross Validation for Logistic Regression

In [ ]:
#tprecision= 0
#trecall= 0
#tf1score1= 0
taccuracy = 0
for i in range(10):
    
    trainrdd, testrdd = training.randomSplit([0.9,0.1], 2*i*i)
    model = LogisticRegressionWithLBFGS.train(trainrdd,numClasses=2)
    labelsAndPred = testrdd.map(lambda p: (float(model.predict(p.features)),p.label))
    metrics = MulticlassMetrics(labelsAndPred)
    x = metrics.confusionMatrix().toArray()
    trueP=x[1][1]
    trueN=x[0][0]
    falseP=x[0][1]
    falseN=x[1][0]
    #tprecision+= trueP/(trueP+falseP)
    #trecall+= trueP/(trueP+falseN)
    #tf1score1+= (2*tprecision*trecall)/(tprecision+trecall)
    taccuracy+= (trueP+trueN)/(trueP+trueN+falseP+falseN)
    print(taccuracy)
accuracy = taccuracy/10
print("Final Accuracy for K-Cross Logistic Regression is: ",accuracy)    

Decision Tree

In [ ]:
from pyspark.mllib.tree import DecisionTree,DecisionTreeModel
model = DecisionTree.trainClassifier(test,numClasses=2, categoricalFeaturesInfo={},maxDepth=5,maxBins=10)
#Can change the 'test' to 'train' to predict accordingly
labelsAndPred = test.map(lambda p: (float(model.predict(p.features)),p.label))


In [ ]:
# Calculating metrics from the confusion matrix
metrics = MulticlassMetrics(labelsAndPred)
x = metrics.confusionMatrix().toArray()
trueP=x[1][1]
trueN=x[0][0]
falseP=x[0][1]
falseN=x[1][0]
dtprecision= trueP/(trueP+falseP)
dtgrecall= trueP/(trueP+falseN)
dtf1score= (2*dtprecision*dtrecall)/(dtprecision+dtrecall)
dtaccuracy = (trueP+trueN)/(trueP+trueN+falseP+falseN)
print (dtprecision,dtrecall,dtf1score,dtaccuracy)

In [ ]:
10-fold Cross Validation for decision tree

In [ ]:
#tprecision= 0
#trecall= 0
#tf1score1= 0
taccuracy = 0
for i in range(10):
    
    trainrdd, testrdd = training.randomSplit([0.9,0.1], 2*i*i)
    model = DecisionTree.trainClassifier(test,numClasses=2, categoricalFeaturesInfo={},maxDepth=5,maxBins=10)
    labelsAndPred = testrdd.map(lambda p: (float(model.predict(p.features)),p.label))
    metrics = MulticlassMetrics(labelsAndPred)
    x = metrics.confusionMatrix().toArray()
    trueP=x[1][1]
    trueN=x[0][0]
    falseP=x[0][1]
    falseN=x[1][0]
    #tprecision+= trueP/(trueP+falseP)
    #trecall+= trueP/(trueP+falseN)
    #tf1score1+= (2*tprecision*trecall)/(tprecision+trecall)
    taccuracy+= (trueP+trueN)/(trueP+trueN+falseP+falseN)
    print(taccuracy)
accuracy = taccuracy/10
print("Final Accuracy for K-Cross Logistic Regression is: ",accuracy) 

Plotting Accuracies

In [ ]:
plt.clf() 
plt.plot([0.849,0.718,0.701],label='Naive Bayes') #Plotting Naive Bayes Accuracies
plt.plot([0.886,0.725,0.775],label='Logistic Regression') #Plotting Logistic regression Accuracies
plt.ylabel("Accuracy")
#Plot with Training accracy at 0.0, 10fold accuracy at 1.0 and Test accuracy at 2.0 on the X-axis
plt.xlabel("Training                                     10fold                                         Test")
plt.legend()
plt.show()
plt.savefig('Accuracies.png') #savefig in home directory if .show() does not work